<a href="https://colab.research.google.com/github/rahiakela/transformers-research-and-practice/blob/main/peft_huggingface_guide/02_lora_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

**Reference**:

https://huggingface.co/docs/peft/v0.9.0/task_guides/lora_based_methods

In [ ]:
!pip install -q peft transformers datasets

In [2]:
from datasets import load_dataset

from transformers import AutoImageProcessor

import torch
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

##Dataset

In [ ]:
ds = load_dataset("food101")

In [4]:
labels = ds["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'baklava'

In [ ]:
# Load an image processor to properly resize and normalize the pixel values
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

In [7]:
# let's prepare some transformation functions for data augmentation and pixel scaling
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

train_transforms = Compose(
    [
        RandomResizedCrop(image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize
    ]
)

val_transforms = Compose(
    [
        Resize(image_processor.size["height"]),
        CenterCrop(image_processor.size["height"]),
        ToTensor(),
        normalize
    ]
)

def preprocess_train(example_batch):
  example_batch["pixel_values"] = [
      train_transforms(image.convert("RGB") for image in example_batch["image"])
  ]
  return example_batch

def preprocess_val(example_batch):
  example_batch["pixel_values"] = [
      val_transforms(image.convert("RGB") for image in example_batch["image"])
  ]
  return example_batch

In [8]:
# let's define the training and validation datasets
train_ds = ds["train"]
val_ds = ds["validation"]

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [9]:
# Finally, need a data collator to create a batch of training and evaluation data
# and convert the labels to torch.tensor objects
def collate_fn(examples):
  pixel_values = torch.stack([example["pixel_values"] for example in examples])
  labels = torch.stack([example["label"] for example in examples])
  return {"pixel_values": pixel_values, "labels": labels}

##Model